In [2]:
import pandas as pd

In [37]:
df1 = pd.read_csv('train_dataset/train_dataset_1.csv')
df2 = pd.read_csv('train_dataset/train_dataset_2.csv')
df3 = pd.read_csv('train_dataset/train_dataset_3.csv')
df4 = pd.read_csv('train_dataset/train_dataset_4.csv')
df5 = pd.read_csv('train_dataset/train_dataset_5.csv')
df6 = pd.read_csv('train_dataset/train_dataset_6.csv')
df7 = pd.read_csv('train_dataset/train_dataset_7.csv')
df8 = pd.read_csv('train_dataset/train_dataset_8.csv')
df9 = pd.read_csv('train_dataset/train_dataset_9.csv')
df10 = pd.read_csv('train_dataset/train_dataset_10.csv')

# df11 = pd.read_csv('test_dataset/test_dataset_1.csv')
# df12 = pd.read_csv('test_dataset/test_dataset_2.csv')
# df13 = pd.read_csv('test_dataset/test_dataset_3.csv')
# df14 = pd.read_csv('test_dataset/test_dataset_4.csv')
# df15 = pd.read_csv('test_dataset/test_dataset_5.csv')
# df16 = pd.read_csv('test_dataset/test_dataset_6.csv')
# df17 = pd.read_csv('test_dataset/test_dataset_7.csv')
# df18 = pd.read_csv('test_dataset/test_dataset_8.csv')
# df19 = pd.read_csv('test_dataset/test_dataset_9.csv')
# df20 = pd.read_csv('test_dataset/test_dataset_10.csv')

Create a column for the current class for all datasets

In [38]:
df1['Class'] = '1'
df2['Class'] = '2'
df3['Class'] = '3'
df4['Class'] = '4'
df5['Class'] = '5'
df6['Class'] = '6'
df7['Class'] = '7'
df8['Class'] = '8'
df9['Class'] = '9'
df10['Class'] = '10'